In [ ]:
import numpy as np
import pandas as pd
import os 
import re

# Clean Data

### Load data and clean up product information

In [ ]:
# Clean data
import os

folder_path = 'path to folder with data'

dfs = [] # list to hold all dataframes prior to concatenation

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dfs.append(df)

product_data_all = pd.concat(dfs)

product_data_all.rename(columns={'Number of Ratings': 'Number_of_Ratings', 'Monthly Purchases': 'Monthly_Purchases', 'Product Information': 'Product_Information'}, inplace=True) # replace spaces with underscores

product_data_all.head()

In [ ]:
product_data_all.shape

In [ ]:
product_data_all.isna().sum()

In [ ]:
product_data_all.isna().any(axis=1).sum()

In [ ]:
# remove unnecessary phrases from Product_Information

product_info_cleaned = []

for i in range(len(product_data_all)):
    if type(product_data_all.iloc[i, 5]) != str:
        product_info_cleaned.append(None)
        continue
    else:
        info = product_data_all.iloc[i, 5]
    
    while "\n" in info or "\u200e" in info or "Product information" in info or "Technical Details" in info or "Additional Information" in info or "out of 5 stars" in info or "Collapse All" in info or "Collapse all" in info or "Expand All" in info or "Measurements" in info or "Item details" in info or "Features & Specs" in info or "User guide" in info or "          Style  " in info or "Materials & Care" in info or "Summary" in info or "Other Technical Details" in info:
        if "\n" in info:
            index = info.find("\n")
            info = info[:index] + info[index+1:]
        if "\u200e" in info:
            index = info.find("\u200e")               
            info = info[:index] + info[index+1:]
        if "Product information" in info:
            index = info.find("Product information")
            info = info[:index] + info[index+20:]
        if "Technical Details" in info:
            index = info.find("Technical Details")
            info = info[:index] + info[index+18:]
        if "Additional Information" in info:
            index = info.find("Additional Information")
            info = info[:index] + info[index+23:]
        if "out of 5 stars" in info:
            indexa = info.find("Customer Reviews")
            matches = re.finditer("out of 5 stars", info)
            positions = [match.start() for match in matches]
            indexb = positions[-1]
            info = info[:indexa] + info[indexb+14:]
        if "Collapse All" in info:
            index = info.find("Collapse All")
            info = info[:index] + info[index+13:]
        if "Collapse all" in info:
            index = info.find("Collapse all")
            info = info[:index] + info[index+13:]
        if "Expand All" in info:
            index = info.find("Expand All")
            info = info[:index] + info[index+11:]
        if "Measurements" in info:
            index = info.find("Measurements")
            info = info[:index] + info[index+13:]
        if "Item details" in info:
            index = info.find("Item details")
            info = info[:index] + info[index+13:]
        if "Features & Specs" in info:
            index = info.find("Features & Specs")
            info = info[:index] + info[index+17:]
        if "User guide" in info:
            index = info.find("User guide")
            info = info[:index] + info[index+11:]
        if "          Style  " in info:
            index = info.find("          Style  ")
            info = info[:index] + info[index+18:]
        if "Materials & Care" in info:
            index = info.find("Materials & Care")
            info = info[:index] + info[index+17:]
        if "Summary" in info:
            index = info.find("Summary")
            info = info[:index] + info[index+8:]
        if "Other Technical Details" in info:
            index = info.find("Other Technical Details")
            info = info[:index] + info[index+24:]
        
    info = info.strip()

    raw_terms = info.split("   ")

    while "" in raw_terms:
        for term in raw_terms:
            if term == "":
                raw_terms.remove(term)

    terms = [term.strip() for term in raw_terms]

    product_info_categories = {}

    indices = list(range(0, len(terms)-1, 2))

    for i in indices:
        if i % 2 == 0:
            renamed_term = terms[i].replace(' ', '_')
            product_info_categories[renamed_term] = terms[i+1]

    product_info_cleaned.append(product_info_categories)

print(len(product_info_cleaned))

In [ ]:
# create new column to hold cleaned product information

product_data_all['Cleaned_Product_Information'] = np.array(product_info_cleaned)
product_data_all.head()

In [ ]:
product_data_all[product_data_all['Cleaned_Product_Information'].isna()]

### Identify product attributes

In [ ]:
# identify unique product attributes in the product information

unique_product_info_attributes = {}

for product in product_info_cleaned:
    if product != None:
        for key, value in product.items():
            if key not in unique_product_info_attributes:
                unique_product_info_attributes[key] = 0
            else:
                unique_product_info_attributes[key] += 1

unique_product_info_count = sorted(unique_product_info_attributes.items(), key=lambda item: item[1], reverse=True)

print(f"{len(unique_product_info_count)} unique product attributes\n")

for key, value in unique_product_info_count:
    print(f"{key}: {value}")

In [ ]:
# create a new column for each unique attribute

new_columns = {}

for category in unique_product_info_attributes:
    temp = []
    
    for i in range(len(product_data_all)):
        product_info = product_data_all['Cleaned_Product_Information'].iloc[i]
        if product_info is None or category not in product_info.keys():
            temp.append(None)
        else:
            temp.append(product_info[category])

    new_columns[category] = temp

new_columns_df = pd.DataFrame(new_columns)

product_data_all = product_data_all.reset_index(drop=True)
new_columns_df = new_columns_df.reset_index(drop=True)

product_data_all = pd.concat([product_data_all, new_columns_df], axis=1)
product_data_all.drop(columns=['Product_Information', 'Cleaned_Product_Information'], inplace=True)

product_data_all.head()

In [ ]:
product_data_all.shape

In [ ]:
# drop duplicate products (each product should have a unqiue ASIN)

product_data_all.dropna(subset=['ASIN'], inplace=True)
product_data_all.drop_duplicates(subset=['ASIN'], keep='first', inplace=True)
product_data_all = product_data_all.reset_index(drop=True)

product_data_all.head()

In [ ]:
product_data_all.shape

In [ ]:
# delete attributes that appear infrequently

attributes_to_delete = []

for i in range(product_data_all.shape[1]):
    if product_data_all.iloc[:, i].count() < 100:
        attributes_to_delete.append(product_data_all.columns[i])

product_data_all.drop(columns=attributes_to_delete, inplace=True)

product_data_all.shape

In [ ]:
# sometimes, a forward slash '/' is used to denote a missing attribute, so we remove them here

for i in range(len(product_data_all)):
    for j in range(product_data_all.shape[1]):
        if product_data_all.iloc[i, j] == '/':
            product_data_all.iloc[i, j] = None

### Clean up original columns

In [ ]:
# clean up Rating, Number_of_Ratings, Monthly_Purchases, and Price

for i in range(len(product_data_all)):
    if type(product_data_all['Rating'][i]) == str and " out of 5 stars" in product_data_all['Rating'][i]:
        index = product_data_all['Rating'][i].find(" out of 5 stars")
        product_data_all['Rating'][i] = product_data_all['Rating'][i][:index]
    else:
        product_data_all['Rating'][i] = None
        
    if type(product_data_all['Number_of_Ratings'][i]) == str and " r" in product_data_all['Number_of_Ratings'][i]:
        index = product_data_all['Number_of_Ratings'][i].find(" r")
        product_data_all['Number_of_Ratings'][i] = product_data_all['Number_of_Ratings'][i][:index]
    if type(product_data_all['Number_of_Ratings'][i]) == str and "," in product_data_all['Number_of_Ratings'][i]:
        while "," in product_data_all['Number_of_Ratings'][i]:
            index = product_data_all['Number_of_Ratings'][i].find(",")
            product_data_all['Number_of_Ratings'][i] = product_data_all['Number_of_Ratings'][i][:index] + product_data_all['Number_of_Ratings'][i][index+1:]
        
    if type(product_data_all['Monthly_Purchases'][i]) == str and "+ bought in past month" in product_data_all['Monthly_Purchases'][i]:
        product_data_all['Monthly_Purchases'][i] = product_data_all['Monthly_Purchases'][i][:-22]
    if type(product_data_all['Monthly_Purchases'][i]) == str and "K" in product_data_all['Monthly_Purchases'][i]:
        product_data_all['Monthly_Purchases'][i] = int(product_data_all['Monthly_Purchases'][i][:-1]) * 1000
    
    if type(product_data_all['Price'][i]) == str and "$" in product_data_all['Price'][i] and "/" not in product_data_all['Price'][i]:
        while "," in product_data_all['Price'][i]:
            index = product_data_all['Price'][i].find(",")
            product_data_all['Price'][i] = product_data_all['Price'][i][:index] + product_data_all['Price'][i][index+1:]
        while "$" in product_data_all['Price'][i]:
            index = product_data_all['Price'][i].find("$")
            product_data_all['Price'][i] = product_data_all['Price'][i][index+1:]
        product_data_all['Price'][i] = product_data_all['Price'][i][1:]
    else:
        product_data_all['Price'][i] = None

product_data_all.head()

In [ ]:
# convert columns into desired data types

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all['Rating'][i]):
        product_data_all['Rating'][i] = float(product_data_all['Rating'][i])

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all['Number_of_Ratings'][i]):
        product_data_all['Number_of_Ratings'][i] = int(product_data_all['Number_of_Ratings'][i])

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all['Monthly_Purchases'][i]):
        product_data_all['Monthly_Purchases'][i] = int(product_data_all['Monthly_Purchases'][i])

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all['Price'][i]):
        product_data_all['Price'][i] = float(product_data_all['Price'][i])

In [ ]:
# download and reload the changes you've made so far, makes debugging easier

product_data_all.to_csv('name of 1st csv', index=False)

### Clean up attributes

#### Take care of repeated information

In [ ]:
product_data_all = pd.read_csv('path to 1st csv', low_memory=False)
product_data_all.head()

In [ ]:
# these attributes are to be deleted because they either have the same information as other attributes or will be combined with other attributes

unwanted_attributes = ['Size', 'Sheet_Size', 'Special_Features', 'Item_Package_Dimensions_L_x_W_x_H', 'Brand_Name', 'Material_Type', 'Product_Dimensions', 
                       'Item_Dimensions_D_x_W_x_H', 'Item_dimensions_L_x_W_x_H', 'Part_Number', 'Finish', 'Finish_types', 'Furniture_Finish',
                       'Batteries_required', 'Number_of_pieces', 'Number_Of_Pieces', 'Shade_Material', 'Top_Material_Type', 'Base_Material', 'Fabric_Type', 
                       'Cover_Material', 'Fill_Material', 'Handle_Material', 'Blade_Material', 'Outer_Material', 'Shade_Color', 'Ink_Color', 'Light_Color', 
                       'Specific_Uses_For_Product']

In [ ]:
# if multiple attributes have the same type of info, consolidate(a, b) transfers all the info to one of the attributes
def consolidate(a, b):
    for i in range(len(product_data_all)):
        if pd.isnull(product_data_all.loc[i, a]) and not pd.isnull(product_data_all.loc[i, b]):
            product_data_all.loc[i, a] = product_data_all.loc[i, b]

# combine(a, b) concatenates the info of string attributes with the same type of info
def combine(a, b):
    for i in range(len(product_data_all)):
        if not pd.isnull(product_data_all.loc[i, a]) and not pd.isnull(product_data_all.loc[i, b]) and product_data_all.loc[i, a] != product_data_all.loc[i, b]:
            a_ = product_data_all.loc[i, a]
            b_ = product_data_all.loc[i, b]
            product_data_all.loc[i, a] = a_ + ", " + b_

In [ ]:
# consolidate attributes with repeat information

consolidate('Special_Feature', 'Special_Features')
consolidate('Package_Dimensions', 'Item_Package_Dimensions_L_x_W_x_H')
consolidate('Brand', 'Brand_Name')
consolidate('Material', 'Material_Type')
consolidate('Item_Dimensions__LxWxH', 'Product_Dimensions')
consolidate('Item_Dimensions__LxWxH', 'Item_Dimensions_D_x_W_x_H')
consolidate('Item_Dimensions__LxWxH', 'Item_dimensions_L_x_W_x_H')
consolidate('Manufacturer_Part_Number', 'Part_Number')
consolidate('Finish_Type', 'Finish')
consolidate('Finish_Type', 'Finish_types')
consolidate('Finish_Type', 'Furniture_Finish')
consolidate('Batteries_Required?', 'Batteries_required')
consolidate('Number_of_Pieces', 'Number_of_pieces')
consolidate('Number_of_Pieces', 'Number_Of_Pieces')

In [ ]:
# combine attributes with same type of information

combine('Material', 'Shade_Material')
combine('Material', 'Top_Material_Type')
combine('Material', 'Base_Material')
combine('Material', 'Fabric_Type')
combine('Material', 'Cover_Material')
combine('Material', 'Fill_Material')
combine('Material', 'Handle_Material')
combine('Material', 'Blade_Material')
combine('Material', 'Outer_Material')
combine('Color', 'Shade_Color')
combine('Color', 'Ink_Color')
combine('Color', 'Light_Color')
combine('Recommended_Uses_For_Product', 'Specific_Uses_For_Product')

In [ ]:
product_data_all.drop(columns=unwanted_attributes, inplace=True)

In [ ]:
# rank attributes by frequency

frequencies = {}

for column in product_data_all.columns:
    frequencies[column] = product_data_all[column].notna().sum()

sorted_frequencies = sorted(frequencies.items(), key=lambda item: item[1], reverse=True)[:50]

top_50_attributes = []

for key, value in sorted_frequencies:
    top_50_attributes.append(key)
    print(f"{key}: {value}")

In [ ]:
# only keep the top 50 most frequent attributes

outside_top_50 = [column for column in product_data_all.columns if column not in top_50_attributes]

product_data_all.drop(columns=outside_top_50, inplace=True)
product_data_all.shape

In [ ]:
# convert the following columns to desired data types

convert_to_int64 = ['Number_of_Ratings', 'Monthly_Purchases', 'Number_of_Items', 'Number_of_Pieces']

for column in convert_to_int64:
    product_data_all[column] = product_data_all[column].astype('Int64')

product_data_all.rename(columns={'Item_Dimensions__LxWxH': 'Item_Dimensions'}, inplace=True) # rename columns with weird names

In [ ]:
# again, download and reload the changes you've made so far, makes debugging easier

product_data_all.to_csv('name of 2nd csv', index=False)

#### Modify data formats

In [ ]:
product_data_all = pd.read_csv('path to 2nd csv', low_memory=False)
product_data_all.head()

In [ ]:
# separate values and units
def value_unit_split(attribute, default_unit=None):
    values = []
    units = []
    index = product_data_all.columns.get_loc(attribute)

    for i in range(len(product_data_all)):
        if not pd.isnull(product_data_all.loc[i, attribute]):
            split = product_data_all.loc[i, attribute].split(' ', maxsplit=1)
            if len(split) == 2:
                values.append(split[0])
                units.append(split[1])
            elif len(split) == 1: 
                if default_unit == None:
                    values.append(None)
                    units.append(None)
                else:
                    values.append(split[0])
                    units.append(default_unit)
        else:
            values.append(None)
            units.append(None)

    # create new columns for values and units
    product_data_all.insert(index+1, f'{attribute}_Value', values)
    product_data_all.insert(index+2, f'{attribute}_Unit', units)

# separate values and units for multidimensional attributes
def dimension_split(attribute):
    dim_length = []
    dim_width = []
    dim_height = []
    dim_units = []
    index = product_data_all.columns.get_loc(attribute)

    for i in range(len(product_data_all)):
        if not pd.isnull(product_data_all.loc[i, attribute]):
            if '"' in product_data_all.loc[i, attribute]: # " means inches
                x = re.sub(r'[a-z]|"', '', product_data_all.loc[i, attribute])
                split = re.findall(r'[\d.]+|[A-Z]+', x)
                
                length = None
                width = None
                height = None
                
                for s in range(len(split)):
                    if split[s] == 'L' or split[s] == 'D':
                        length = split[s-1]
                    if split[s] == 'W':
                        width = split[s-1]
                    if split[s] == 'H' or split[s] == 'T':
                        height = split[s-1]
                
                dim_length.append(length)
                dim_width.append(width)
                dim_height.append(height)
                dim_units.append('inches')
            
            else: 
                split = re.findall(r'[\d.]+|[a-zA-Z]+', product_data_all.loc[i, attribute])
                
                dim_length.append(split[0])
                
                if len(split) >= 4:
                    dim_width.append(split[2])
                else:
                    dim_width.append(None)
                
                if len(split) >= 6:
                    dim_height.append(split[4])
                else:
                    dim_height.append(None)
                
                dim_units.append(split[-1])     
        
        else:
            dim_length.append(None)
            dim_width.append(None)
            dim_height.append(None)
            dim_units.append(None)

    # create new columns for values and units
    product_data_all.insert(index+1, f'{attribute}_Length', dim_length)
    product_data_all.insert(index+2, f'{attribute}_Width', dim_width)
    product_data_all.insert(index+3, f'{attribute}_Height', dim_height)
    product_data_all.insert(index+4, f'{attribute}_Units', dim_units)

# split strings on specified delimiters
def delim_split(attribute, to_remove=[None]):
    phrases = set()

    for i in range(len(product_data_all)):
        if not pd.isnull(product_data_all.loc[i, attribute]):
            split = re.split(r'\s*[,，、;\|/]\s*', product_data_all.loc[i, attribute])
            cleaned_split = [s for s in split if any(char.isalnum() for char in s) and not (s.replace(' ', '').isdigit() or (('(' in s and ')' not in s) or ('(' not in s and ')' in s)))]
            for phrase in cleaned_split:
                phrases.add(phrase.strip().lower())

    enum_df = pd.DataFrame({attribute: list(phrases)})
    enum_df = enum_df[~enum_df[attribute].isin(to_remove)] # remove meaningless rows
    enum_df = enum_df.reset_index(drop=True)
    enum_df.insert(0, 'id', list(range(1, len(enum_df)+1))) # create ids
    
    return enum_df

# enumerate attributes if needed
def enum_attr(attribute, enum_df):
    new_column = []

    for i in range(len(product_data_all)):
        if not pd.isnull(product_data_all.loc[i, attribute]):
            ids = set()
            split = re.split(r'\s*[,，、;\|/]\s*', product_data_all.loc[i, attribute].lower())
            for j in range(len(enum_df)):
                for s in split:
                    if enum_df.loc[j, attribute] in s:
                        ids.add(enum_df.loc[j, 'id'])
            if len(ids) == 0:
                new_column.append(None)
            else:
                new_column.append(ids)
        else:
            new_column.append(None)

    return new_column

In [ ]:
# Item_Weight 

value_unit_split('Item_Weight')

index = product_data_all.columns.get_loc('Item_Weight')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Item_Weight']):
        print(product_data_all.iloc[i, index:index+3])

In [ ]:
# Item_Dimensions

dimension_split('Item_Dimensions')

index = product_data_all.columns.get_loc('Item_Dimensions')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Item_Dimensions']):
        print(product_data_all.iloc[i, index:index+5])

In [ ]:
# Color 

pattern = ['multicolor', 
           'black', 'light.*black', 'dark.*black', 'clear.*black', 'matte.*black', 'crystal.*black', 'bordeaux.*black', 'gloss.*black', 'charcoal.*black', 'lavastone.*black', 'bulk.*black',
                'midnight.*black', 'classic.*black', 'cream.*black', 'new.*black', 'pure.*black', 'gloss.*black',
           'white', 'light.*white', 'clear.*white', 'matte.*white', 'neutral.*white', 'snow.*white', 'cream.*white', 'gloss.*white', 'cold.*white', 'cool.*white', 'royal.*white',
                'natural.*white', 'granite.*white',  'flat.*white', 'vintage.*white', 'forest.*white', 'smart.*white', 'frosted.*white', 'warm.*white', 'studio.*white', 'navy.*white',
                'medium.*white', 'new.*white', 'cardinal.*white', 'classic.*white', 'off.*white', 'pure.*white', 'distressed.*white', 'milky.*white', 'weathered.*white', 'stripe.*white',
           'gr[ae]y', 'light.*gr[ae]y', 'matte.*gr[ae]y',  'charcoal.*gr[ae]y', 'dark.*gr[ae]y', 'earthy.*gr[ae]y', 'sea.*salt.*gr[ae]y', 'oyster.*gr[ae]y', 'elegant.*gr[ae]y', 
                'dove.*gr[ae]y', 'slate.*gr[ae]y', 'gun.*gr[ae]y', 'tweed.*gr[ae]y', 'modern.*gr[ae]y', 'antique.*gr[ae]y','stylish.*gr[ae]y', 'fog.*gr[ae]y', 'stainless.*steel.*gr[ae]y',
                'space.*gr[ae]y', 'plush.*gr[ae]y', 
           'brown', 'light.*brown', 'matte.*brown', 'rustic.*brown', 'vintage.*brown', 'deep.*brown', 'berry.*brown', 'vintage.*brown', 'brushed.*brown', 'teak.*brown', 'whiskey.*brown',
                'dark.*brown', 'retro.*brown', 'speckled.*brown', 'craft.*brown', 'stainless.*brown', 
           'red', 'light.*red', 'matte.*red', 'empire.*red', 'transparent.*red.', 'rose.*red', 'dark.*red', 'fire.*engine.*red', 'wine.*red',
           'orange', 'light.*orange', 'matte.*orange', 'fluorescent.*orange', 'burnt.*orange', 'autumn.*orange',
           'yellow', 'light.*yellow', 'matte.*yellow', 'vibrant.*yellow', 'warm.*yellow', 'optic.*yellow', 'canary.*yellow', 
           'green', 'light.*green', 'matte.*green', 'forest.*green', 'off.*green', 'luscious.*green', 'pastel.*green', 'grass.*green', 'matcha.*green', 'mint.*green', 'olive.*green', 
                'dark.*green', 'navy.*green', 'natural.*green', 'neon.*green', 'avacado.*green', 'apple.*green', 'army.*green',
           'blue', 'light.*blue', 'matte.*blue', 'classic.*blue', 'midnight.*blue', 'marine.*blue', 'royal.*blue', 'arctic.*blue', 'baby.*blue', 'adriatic.*blue', 'starry.*blue',
                'bright.*blue', 'neon.*blue', 'nav.*blue', 'cobalt.*blue', 'columbia.*blue', 'coastal.*blue', 'lake.*blue', 'tranquil.*blue', 'slate.*blue', 'pacific.*blue',
                'venom.*blue', 'dark.*blue', 'new.*blue', 'sky.*blue', 'ocean.*blue', 'pure.*blue', 
           'purple', 'light.*purple', 'dark.*purple', 'charming.*purple', 'translucent.*purple', 'deep.*purple',
           'pink', 'light.*pink', 'apricot.*pink', 'pretty.*pink', 'dusty.*pink', 'matte.*pink', 'velvet.*pink', 'neon.*pink', 'bright.*pink', 'pastel.*pink', 'hot.*pink', 'rosy.*pink',
           'gold', 'light.*gold', 'rose.*gold', 'vegas.*gold', 'champaign.*gold', 'white.*gold', 'holographic.*gold.',
           'silver', 'light.*silver',  'vintage.*silver', 'ancient.*silver', 'metallic.*silver', 'star.*silver', 'premium.*silver', 'daylight.*silver', 'matte.*silver', 'crystal.*silver',
                'cloud.*silver',
           'nickel', 'brush.*nickel', 'sand.*nickel', 'matte.*nickel', 'beige', 'light.*beige', 'washed.*beige', 'cream.*beige', 'cherry', 'light.*cherry',
           'wood',  'rustic.*wood', 'burlywood', 'driftwood', 'barnwood', 'flatwood', 'wood.*grain', 'natural.*wood', 'cherry.*wood', 'red.*wood', 'dark.*wood', 
           'pistachio', 'sunflower', 'lime', 'electric lime', 'marble', 'white.*marble', 'gr[ae]y.*marble', 'black.*marble', 'gold.*marble', 'pink.*marble', 
           'onyx', 'oat', 'light.*oat', 'tropical', 'burgundy', 'neon', 'mahogany', 'rose', 'dusty rose', 'morandi', 'quartz', 'teal', 'charcoal',
           'watercolour', 'violet', 'maroon', 'sugar', 'snow', 'honey', 'vintage.*indigo', 'rainbow', 'celeste', 'macaron', 'metallic', 'alabaster', 'frosted.*alabaster',
           'boho', 'chic', 'champagne.*bronze', 'noble.*bronze', 'premier.*bronze', 'old.*bronze', 'aged.*bronze', 'vibrant', 'camel', 'tan', 'scarlet', 'matte.*scarlet', 'citrus',
           'cream', 'lemon', 'tangerine', 'garden', 'light.*garden', 'secret.*garden', 'cement', 'peach', 'coastal', 'chrome', 'hot', 'titanium', 'antique.*pewter', 'turquoise',
           'sandstone', 'cyan', 'black.*oak', 'forest', 'raspberry', 'greige', 'royal', 'velvet', 'blueberry', 'mud', 'anvil.*iron', 'indigo', 'gloss.*navy', 
           'espresso', 'copper', 'mojo', 'smoked.*iron', 'emerald', 'amber', 'ivory', 'azure', 'tiger', 'cheetah', 'passion.*maroon',
           'smoke', 'coral', 'day.*light', 'peacock', 'pastel', 'mountain.*haze', 'flaxen', 'cotton.*candy', 'terrazzo', 'jewel', 'tumbleweed',
           'clover', 'lavender', 'floral', 'transparent', 'white.*sand', 'blue.*lightning', 'flower', 'eucalyptus', 'navy', 'wild.*berry', 'leopard', 'marigold',
           'blush', 'khaki', 'crystal', 'elegant', 'coffee.*haze', 'polished.*brass',
           'sage', 'manilla', 'mocha', 'strawberry', 'pearl', 'neutral', 'blonde', 'acacia', 'camo', 'ocean', 'cardinal', 'aqua', 'gradient', 'natural', 'brass', 'space',
           'walnut', 'white.*walnut', 'dark.*walnut', 'american.*walnut', 'columbia.*walnut']

color = ['multicolor', 
           'black', 'light black', 'dark black', 'clear black', 'matte black', 'crystal black', 'bordeaux black', 'gloss black', 'charcoal black', 'lavastone black', 'bulk black',
                'midnight black', 'classic black', 'cream black', 'new black', 'pure black', 'gloss black',
           'white', 'light white', 'clear white', 'matte white', 'neutral white', 'snow white', 'creamy white', 'gloss white', 'cold white', 'cool white', 'royal white',
                'natural white', 'granite white',  'flat white', 'vintage white', 'forest white', 'smart white', 'frosted white', 'warm white', 'studio white', 'navy white',
                'medium white', 'new white', 'cardinal white', 'classic white', 'off white', 'pure white', 'distressed white', 'milky white', 'weathered white', 'stripe white',
           'gray', 'light gray', 'matte gray',  'charcoal gray', 'dark gray', 'earthy gray', 'sea salt gray', 'oyster gray', 'elegant gray', 
                'dove gray', 'slate gray', 'gun gray', 'tweed gray', 'modern gray', 'antique gray','stylish gray', 'fog gray', 'stainless steel gray',
                'space gray', 'plush gray', 
           'brown', 'light brown', 'matte brown', 'rustic brown', 'vintage brown', 'deep brown', 'berry brown', 'vintage brown', 'brushed brown', 'teak brown', 'whiskey brown',
                'dark brown', 'retro brown', 'speckled brown', 'craft brown', 'stainless brown', 
           'red', 'light red', 'matte red', 'empire red', 'transparent red.', 'rose red', 'dark red', 'fire engine red', 'wine red',
           'orange', 'light orange', 'matte orange', 'fluorescent orange', 'burnt orange', 'autumn orange',
           'yellow', 'light yellow', 'matte yellow', 'vibrant yellow', 'warm yellow', 'optic yellow', 'canary yellow', 
           'green', 'light green', 'matte green', 'forest green', 'off green', 'luscious green', 'pastel green', 'grass green', 'matcha green', 'mint green', 'olive green', 
                'dark green', 'navy green', 'natural green', 'neon green', 'avacado green', 'apple green', 'army green',
           'blue', 'light blue', 'matte blue', 'classic blue', 'midnight blue', 'marine blue', 'royal blue', 'arctic blue', 'baby blue', 'adriatic blue', 'starry blue',
                'bright blue', 'neon blue', 'navy blue', 'cobalt blue', 'columbia blue', 'coastal blue', 'lake blue', 'tranquil blue', 'slate blue', 'pacific blue',
                'venom blue', 'dark blue', 'new blue', 'sky blue', 'ocean blue', 'pure blue', 
           'purple', 'light purple', 'dark purple', 'charming purple', 'translucent purple', 'deep purple',
           'pink', 'light pink', 'apricot pink', 'pretty pink', 'dusty pink', 'matte pink', 'velvet pink', 'neon pink', 'bright pink', 'pastel pink', 'hot pink', 'rosy pink',
           'gold', 'light gold', 'rose gold', 'vegas gold', 'champaign gold', 'white gold', 'holographic gold.',
           'silver', 'light silver', 'vintage silver', 'ancient silver', 'metallic silver', 'star silver', 'premium silver', 'daylight silver', 'matte silver', 'crystal silver',
                'cloud silver',
           'nickel', 'brush nickel', 'sand nickel', 'matte nickel', 'beige', 'light beige', 'washed beige', 'cream beige', 'cherry', 'light cherry',
           'wood',  'rustic wood', 'burlywood', 'driftwood', 'barnwood', 'flatwood', 'wood grain', 'natural wood', 'cherry wood', 'redwood', 'dark wood', 
           'pistachio', 'sunflower', 'lime', 'electric lime', 'marble', 'white marble', 'gray marble', 'black marble', 'gold marble', 'pink marble', 
           'onyx', 'oat', 'light oat', 'tropical', 'burgundy', 'neon', 'mahogany', 'rose', 'dusty rose', 'morandi', 'quartz', 'teal', 'charcoal',
           'watercolour', 'violet', 'maroon', 'sugar', 'snow', 'honey', 'vintage indigo', 'rainbow', 'celeste', 'macaron', 'metallic', 'alabaster', 'frosted alabaster',
           'boho', 'chic', 'champagne bronze', 'noble bronze', 'premier bronze', 'old bronze', 'aged bronze','vibrant', 'camel', 'tan', 'scarlet', 'matte scarlet', 'citrus',
           'cream', 'lemon', 'tangerine', 'garden', 'light garden', 'secret garden', 'cement', 'peach', 'coastal', 'chrome', 'hot', 'titanium', 'antique pewter', 'turquoise',
           'sandstone', 'cyan', 'black oak', 'forest', 'raspberry', 'greige', 'royal', 'velvet', 'blueberry', 'mud', 'anvil iron', 'indigo', 'gloss navy', 
           'espresso', 'copper', 'mojo', 'smoked iron', 'emerald', 'amber', 'ivory', 'azure', 'tiger', 'cheetah', 'passion maroon',
           'smoke', 'coral', 'daylight', 'peacock', 'pastel', 'mountain haze', 'flaxen', 'cotton candy', 'terrazzo', 'jewel', 'tumbleweed',
           'clover', 'lavender', 'floral', 'transparent', 'white sand', 'blue lightning', 'flower', 'eucalyptus', 'navy', 'wild berry', 'leopard', 'marigold',
           'blush', 'khaki', 'crystal', 'elegant', 'coffee haze', 'polished brass',
           'sage', 'manilla', 'mocha', 'strawberry', 'pearl', 'neutral', 'blonde', 'acacia', 'camo', 'ocean', 'cardinal', 'aqua', 'gradient', 'natural', 'brass', 'space',
           'walnut', 'white walnut', 'dark walnut', 'american walnut', 'columbia walnut']

color_enum = pd.DataFrame({'id': list(range(1, len(color)+1)), 'pattern': pattern, 'Color': color})

color_new = []

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Color']): 
        ids = set()
        split = re.split(r'\s*[,，、+;\|/]\s*', product_data_all.loc[i, 'Color'].lower())
        for j in range(len(color_enum)):
            for s in split:
                if 'assort' in s or 'variety' in s or 'multi' in s or 'mix' in s or 'adjustable' in s:
                    ids.add(1)
                if bool(re.search(color_enum.loc[j, 'pattern'], s)):
                    ids.add(color_enum.loc[j, 'id'])
        if len(ids) == 0:
            color_new.append(None)
        else:
            color_new.append(ids)
    else:
        color_new.append(None)

color_enum.drop(columns='pattern', inplace=True)
color_enum.to_csv('/Users/fei/Desktop/Mixer/Color_enum.csv', index=False)

product_data_all['Color'] = color_new

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Color']):
        print(product_data_all.loc[i, 'Color'])

In [ ]:
# Style 

redundant = ['no', '2 - 7 1', 'm', '-11', '4 - 6 7', '8 - 7 1', '8" - 7 1']

style_enum = delim_split('Style', redundant)

product_data_all['Style'] = enum_attr('Style', style_enum)

style_enum.to_csv('/Users/fei/Desktop/Mixer/Style_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Style']):
        print(product_data_all.loc[i, 'Style'])

In [ ]:
# Room_Type 

redundant = ['etc', 'di', 'etc.', 'office.', 'dinning room', 'living room dining room', 'dining room living room', 'nursery room', 'hallways', 'kitchen room', 'liberary', 'nursary', 'bedoom', 'sturdy room', 
             'homehouse', 'librar', 'dinning rooms', "kid's room", 'kids room', 'kids room nursery', 'kids', 'bedroom dining room dorm room home office kids room living room nursery playroom study room',
             'kid room', 'bathroom bedroom dining room dorm room home office kids room kitchen laundry room living room', "kids' room", "kid’s room", 
             'bedroom dining room dorm room home ofice kids room kitchen laundry room', 'kids nursery', 'bedroom living room dining room kids room girls room boys room kitchen home office', 'multiple room types',
             'multiple rooms', 'playroom', 'play room', 'living room nursery playroom study room', 'study reading room', 'study room', 'living room', 'bedroom kitchen island living room dinning room home office',
             'living rooms', 'bedroom living room', 'bathroom living room', 'livingroom', 'living room bedroom', 'incl. living room', 'office bedroom', 'bedroom and office', 'socket ceiling fan for bedroom',
             'indoor for bedroom', 'bedroom kitchen dining room', 'indoor ceiling fans for bedroom', 'dining room nl bedroom', 'bedrooms', 'dining room', 
             'high cfm ceiling fan for exterior interior dining room family room', 'diningroom', 'office room', 'office-9w', 'office island aisle hotel room cafe school', 'home or oudoor'
             'modern farmhouse ceiling fan for high medium low sloped ceiling', '3 blade commercial industrial residential ceiling fan with light', 'patios', 'outdoor ceiling fans for patios', 'patio garden',
             'outdoor patio', 'and more.', 'indoor places', 'office and other indoor space', 'coverd outdoor', 'outdoors', 'outdoor for porch', 'covered outdoor areas']

room_type_enum = delim_split('Room_Type', redundant)

room_type_enum.loc[len(room_type_enum)] = [len(room_type_enum)+1, 'kid']
room_type_enum.loc[len(room_type_enum)] = [len(room_type_enum)+1, 'play']
room_type_enum.loc[len(room_type_enum)] = [len(room_type_enum)+1, 'dining']

product_data_all['Room_Type'] = enum_attr('Room_Type', room_type_enum)

room_type_enum.to_csv('/Users/fei/Desktop/Mixer/Room_Type_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Room_Type']):
        print(product_data_all.loc[i, 'Room_Type'])

In [ ]:
# Special_Feature 

redundant = ['enc', 'n\\\\a', 'etc.', "'foldable']", "['portable'"]

special_feature_enum = delim_split('Special_Feature', redundant)

product_data_all['Special_Feature'] = enum_attr('Special_Feature', special_feature_enum)

special_feature_enum.to_csv('/Users/fei/Desktop/Mixer/Special_Feature_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Special_Feature']):
        print(product_data_all.loc[i, 'Special_Feature'])

In [ ]:
# Recommended_Uses_For_Product 

redundant = ['no', 'etc', 'etc.']

Recommended_Uses_For_Product_enum = delim_split('Recommended_Uses_For_Product', redundant)

product_data_all['Recommended_Uses_For_Product'] = enum_attr('Recommended_Uses_For_Product', Recommended_Uses_For_Product_enum)

Recommended_Uses_For_Product_enum.to_csv('/Users/fei/Desktop/Mixer/Recommended_Uses_For_Product_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Recommended_Uses_For_Product']):
        print(product_data_all.loc[i, 'Recommended_Uses_For_Product'])

In [ ]:
# Wattage 

value_unit_split('Wattage', 'watts')

index = product_data_all.columns.get_loc('Wattage')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Wattage']):
        print(product_data_all.iloc[i, index:index+3])

In [ ]:
# Voltage 

value_unit_split('Voltage', 'Volts')

index = product_data_all.columns.get_loc('Voltage')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Voltage']):
        print(product_data_all.iloc[i, index:index+3])

In [ ]:
# Switch_Type 

redundant = ['remote contrl', 'remote control', 'off switch', 'push button', 'vocie &app control', 'smartphone app', 'voice command', 'off switch', 'push button or app', 'dimmer switch', 'on-off knob switch', 
             'app control', 'rotary or push switch', 'rotary knob switch', 'rotary dial', 'rotary & touch', 'knob control', '3 speed pull chain', 'push button" or "slide', 'ceiling fans with lights and remote control', 
             'touchpad', 'remote & foot switch & phone app', 'remote and wall switch', 'touch control & romote control', 'buttons', 'foot switch', 'basic on']

Switch_Type_enum = delim_split('Switch_Type', redundant)

product_data_all['Switch_Type'] = enum_attr('Switch_Type', Switch_Type_enum)

Switch_Type_enum.to_csv('/Users/fei/Desktop/Mixer/Switch_Type_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Switch_Type']):
        print(product_data_all.loc[i, 'Switch_Type'])

In [ ]:
# Included_Components 

redundant = ['no', 'non', '0.1', 'dw']

Included_Components_enum = delim_split('Included_Components', redundant)

product_data_all['Included_Components'] = enum_attr('Included_Components', Included_Components_enum)

Included_Components_enum.to_csv('/Users/fei/Desktop/Mixer/Included_Components_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Included_Components']):
        print(product_data_all.loc[i, 'Included_Components'])

In [ ]:
# Indoor/Outdoor_Usage 

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Indoor/Outdoor_Usage']):
        if product_data_all.loc[i, 'Indoor/Outdoor_Usage'] == 'Outdoor, Indoor':
            product_data_all.loc[i, 'Indoor/Outdoor_Usage'] = 'Both'

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Indoor/Outdoor_Usage']):
        print(product_data_all.loc[i, 'Indoor/Outdoor_Usage'])

In [ ]:
# Control_Method 

Control_Method_enum = delim_split('Control_Method')

product_data_all['Control_Method'] = enum_attr('Control_Method', Control_Method_enum)

Control_Method_enum.to_csv('/Users/fei/Desktop/Mixer/Control_Method_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Control_Method']):
        print(product_data_all.loc[i, 'Control_Method'])

In [ ]:
# Best_Sellers_Rank 

first_rank = []
first_category = []
second_rank = []
second_category = []
third_rank = []
third_category = []
fourth_rank = []
fourth_category = []

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Best_Sellers_Rank']):
        split = [term.strip() for term in product_data_all.loc[i, 'Best_Sellers_Rank'].split('#')]
        split = [term for term in split if term != '']
        sep = [s.split('in', maxsplit=1) for s in split]
        for section in sep:
            section[0] = section[0].replace(',', '').strip()
            ind = section[1].find('(')
            section[1] = section[1][:ind].strip()
        first_rank.append(sep[0][0])
        first_category.append(sep[0][1])

        sr, sc, tr, tc, fr, fc = None, None, None, None, None, None
        
        if len(sep) > 1:
            sr = sep[1][0]
            sc = sep[1][1]
        if len(sep) > 2:
            tr = sep[2][0]
            tc = sep[2][1]
        if len(sep) > 3:
            fr = sep[3][0]
            fc = sep[3][1]
            
        second_rank.append(sr)
        second_category.append(sc)
        third_rank.append(tr)
        third_category.append(tc)
        fourth_rank.append(fr)
        fourth_category.append(fc)
        
    else:
        first_rank.append(None)
        first_category.append(None)
        second_rank.append(None)
        second_category.append(None)
        third_rank.append(None)
        third_category.append(None)
        fourth_rank.append(None)
        fourth_category.append(None)

index = product_data_all.columns.get_loc('Best_Sellers_Rank')

product_data_all.insert(index+1, 'Best_Sellers_First_Rank', first_rank)
product_data_all.insert(index+2, 'Best_Sellers_First_Category', first_category)
product_data_all.insert(index+3, 'Best_Sellers_Second_Rank', second_rank)
product_data_all.insert(index+4, 'Best_Sellers_Second_Category', second_category)
product_data_all.insert(index+5, 'Best_Sellers_Third_Rank', third_rank)
product_data_all.insert(index+6, 'Best_Sellers_Third_Category', third_category)
product_data_all.insert(index+7, 'Best_Sellers_Fourth_Rank', fourth_rank)
product_data_all.insert(index+8, 'Best_Sellers_Fourth_Category', fourth_category)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Best_Sellers_Rank']):
        print(product_data_all.iloc[i, index:index+9])

In [ ]:
# Material 

redundant = ['other-materials']

Material_enum = delim_split('Material', redundant)

product_data_all['Material'] = enum_attr('Material', Material_enum)

Material_enum.to_csv('/Users/fei/Desktop/Mixer/Material_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Material']):
        print(product_data_all.loc[i, 'Material'])

In [ ]:
# Theme 

Theme_enum = delim_split('Theme')

product_data_all['Theme'] = enum_attr('Theme', Theme_enum)

Theme_enum.to_csv('/Users/fei/Desktop/Mixer/Theme_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Theme']):
        print(product_data_all.loc[i, 'Theme'])

In [ ]:
# Shape 

redundant = ['TRUE']

Shape_enum = delim_split('Shape', redundant)

product_data_all['Shape'] = enum_attr('Shape', Shape_enum)

Shape_enum.to_csv('/Users/fei/Desktop/Mixer/Shape_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Shape']):
        print(product_data_all.loc[i, 'Shape'])

In [ ]:
# Mounting_Type 

Mounting_Type_enum = delim_split('Mounting_Type')

product_data_all['Mounting_Type'] = enum_attr('Mounting_Type', Mounting_Type_enum)

Mounting_Type_enum.to_csv('/Users/fei/Desktop/Mixer/Mounting_Type_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Mounting_Type']):
        print(product_data_all.loc[i, 'Mounting_Type'])

In [ ]:
# Finish_Type 

redundant = ['painted finish', 'painted or powder coated', 'hand painted', 'painted or plastic coated', 'painted]']

Finish_Type_enum = delim_split('Finish_Type', redundant)

product_data_all['Finish_Type'] = enum_attr('Finish_Type', Finish_Type_enum)

Finish_Type_enum.to_csv('/Users/fei/Desktop/Mixer/Finish_Type_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Finish_Type']):
        print(product_data_all.loc[i, 'Finish_Type'])

In [ ]:
# Unit_Count

value_unit_split('Unit_Count', 'Count')
product_data_all['Unit_Count_Value'] = product_data_all['Unit_Count_Value'].astype(float).round().astype('Int64')

index = product_data_all.columns.get_loc('Unit_Count')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Unit_Count']):
        print(product_data_all.iloc[i, index:index+3])

In [ ]:
# Package_Dimensions 

dimension_split('Package_Dimensions')

index = product_data_all.columns.get_loc('Package_Dimensions')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Package_Dimensions']):
        print(product_data_all.iloc[i, index:index+5])

In [ ]:
# Pattern 

redundant = ['non', 'no', 'fans']

Pattern_enum = delim_split('Pattern', redundant)

product_data_all['Pattern'] = enum_attr('Pattern', Pattern_enum)

Pattern_enum.to_csv('/Users/fei/Desktop/Mixer/Pattern_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Pattern']):
        print(product_data_all.loc[i, 'Pattern'])

In [ ]:
# Product_Care_Instructions 

redundant = ['no']

Product_Care_Instructions_enum = delim_split('Product_Care_Instructions', redundant)

product_data_all['Product_Care_Instructions'] = enum_attr('Product_Care_Instructions', Product_Care_Instructions_enum)

Product_Care_Instructions_enum.to_csv('/Users/fei/Desktop/Mixer/Product_Care_Instructions_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Product_Care_Instructions']):
        print(product_data_all.loc[i, 'Product_Care_Instructions'])

In [ ]:
# Package_Weight 

value_unit_split('Package_Weight')

index = product_data_all.columns.get_loc('Package_Weight')

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Package_Weight']):
        print(product_data_all.iloc[i, index:index+3])

In [ ]:
# Suggested_Users 

redundant = ['unisex-teen', 'unisex-adult unisex-child mens womens', 'coolflo batting helmet cfabhn', 'batting helmets', 'unisex adult&youth', 'unisex_youth', 'mens', 'adults', 'unisex-adult', 'unisex-youth',
             'unisex youth', 'adult & teenager', 'unisex adult', 'unisex-child', 'z5 solid senior batting helmet']

Suggested_Users_enum = delim_split('Suggested_Users', redundant)

product_data_all['Suggested_Users'] = enum_attr('Suggested_Users', Suggested_Users_enum)

Suggested_Users_enum.to_csv('/Users/fei/Desktop/Mixer/Suggested_Users_enum.csv', index=False)

for i in range(len(product_data_all)):
    if not pd.isnull(product_data_all.loc[i, 'Suggested_Users']):
        print(product_data_all.loc[i, 'Suggested_Users'])

In [ ]:
# drop the columns from which we extracted and separated values and units

product_data_all.drop(columns=['Item_Weight', 'Item_Dimensions', 'Wattage', 'Voltage', 'Best_Sellers_Rank', 'Unit_Count', 'Package_Dimensions', 'Package_Weight'], inplace=True)

In [ ]:
# convert the following columns to desired data types

convert_to_int64 = ['Number_of_Ratings', 'Monthly_Purchases', 'Number_of_Items', 'Number_of_Pieces']

for column in convert_to_int64:
    product_data_all[column] = product_data_all[column].astype('Int64')

In [ ]:
product_data_all.to_csv('name of final csv', index=False)